In [51]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import requests
from tqdm import tqdm

In [2]:
movies = pd.read_csv("tmdb_movies_final.csv")
credit = pd.read_csv("tmdb_credits_final.csv")

In [3]:
movies

,id,title,overview,genres,keywords
0,13499,"Yours, Mine & Ours",Admiral Frank Beardsley returns to New London ...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10751, ""...","[{""id"": 4498, ""name"": ""big family""}, {""id"": 18..."
1,411,"The Chronicles of Narnia: The Lion, the Witch ...","Siblings Lucy, Edmund, Susan and Peter step th...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 10751...","[{""id"": 616, ""name"": ""witch""}, {""id"": 6917, ""n..."
2,3933,Corpse Bride,"In a 19th-century European village, a young ma...","[{""id"": 10749, ""name"": ""Romance""}, {""id"": 14, ...","[{""id"": 4062, ""name"": ""skeleton""}, {""id"": 128,..."
3,9738,Fantastic Four,"During a space voyage, four scientists are alt...","[{""id"": 28, ""name"": ""Action""}, {""id"": 14, ""nam...","[{""id"": 2546, ""name"": ""mask""}, {""id"": 6054, ""n..."
4,674,Harry Potter and the Goblet of Fire,When Harry Potter's name emerges from the Gobl...,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 616, ""name"": ""witch""}, {""id"": 3737, ""n..."
...,...,...,...,...,...
115025,1419229,Silent Zone,"In a zombie-ravaged world, a resourceful teen ...","[{""id"": 27, ""name"": ""Horror""}, {""id"": 53, ""nam...","[{""id"": 186565, ""name"": ""zombie apocalypse""}, ..."
115026,1010756,The Strangers: Chapter 2,On the final day of their cross country road t...,"[{""id"": 27, ""name"": ""Horror""}]","[{""id"": 9663, ""name"": ""sequel""}]"
115027,1261148,The Deceased,"When their father dies on a cursed day, terror...","[{""id"": 27, ""name"": ""Horror""}, {""id"": 53, ""nam...","[{""id"": 4720, ""name"": ""ritual""}, {""id"": 6152, ..."
115028,1206083,Conjuring Tapes,While sorting through their late friend's belo...,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 9648, ""n...","[{""id"": 163053, ""name"": ""found footage""}]"


In [4]:
movies = movies.merge(credit,on='title')

In [5]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [6]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,13499,"Yours, Mine & Ours",Admiral Frank Beardsley returns to New London ...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10751, ""...","[{""id"": 4498, ""name"": ""big family""}, {""id"": 18...","[{""cast_id"": 1, ""character"": ""Frank Beardsley""...","[{""credit_id"": ""52fe45729251416c75056cbb"", ""de..."
1,13499,"Yours, Mine & Ours",Admiral Frank Beardsley returns to New London ...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10751, ""...","[{""id"": 4498, ""name"": ""big family""}, {""id"": 18...","[{""cast_id"": 1, ""character"": ""Frank Beardsley""...","[{""credit_id"": ""52fe45729251416c75056cbb"", ""de..."
2,13499,"Yours, Mine & Ours",Admiral Frank Beardsley returns to New London ...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10751, ""...","[{""id"": 4498, ""name"": ""big family""}, {""id"": 18...","[{""cast_id"": 1, ""character"": ""Frank Beardsley""...","[{""credit_id"": ""52fe45729251416c75056cbb"", ""de..."
3,13499,"Yours, Mine & Ours",Admiral Frank Beardsley returns to New London ...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10751, ""...","[{""id"": 4498, ""name"": ""big family""}, {""id"": 18...","[{""cast_id"": 1, ""character"": ""Frank Beardsley""...","[{""credit_id"": ""52fe45729251416c75056cbb"", ""de..."
4,13499,"Yours, Mine & Ours",Admiral Frank Beardsley returns to New London ...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10751, ""...","[{""id"": 4498, ""name"": ""big family""}, {""id"": 18...","[{""cast_id"": 1, ""character"": ""Frank Beardsley""...","[{""credit_id"": ""52fe45729251416c75056cbb"", ""de..."


In [7]:
movies.dropna(inplace=True)
movies.drop_duplicates(subset='movie_id',inplace=True)
movies.reset_index(drop=True,inplace=True)

In [8]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L
    #converting crazy genre format to list simple format

In [9]:
movies['genres']=movies['genres'].apply(convert)

In [10]:
movies['keywords']=movies['keywords'].apply(convert)

In [11]:
movies.head(1000
        )

,movie_id,title,overview,genres,keywords,cast,crew
0,13499,"Yours, Mine & Ours",Admiral Frank Beardsley returns to New London ...,"[Comedy, Family, Romance]","[big family, family, blended family]","[{""cast_id"": 1, ""character"": ""Frank Beardsley""...","[{""credit_id"": ""52fe45729251416c75056cbb"", ""de..."
1,411,"The Chronicles of Narnia: The Lion, the Witch ...","Siblings Lucy, Edmund, Susan and Peter step th...","[Adventure, Family, Fantasy]","[witch, epic, sibling relationship, saving the...","[{""cast_id"": 5, ""character"": ""Peter Pevensie"",...","[{""credit_id"": ""52fe4240c3a36847f800fd8b"", ""de..."
2,3933,Corpse Bride,"In a 19th-century European village, a young ma...","[Romance, Fantasy, Animation]","[skeleton, love triangle, cheating, wedding vo...","[{""cast_id"": 2, ""character"": ""Victor Van Dort ...","[{""credit_id"": ""52fe43a7c3a36847f8065087"", ""de..."
3,9738,Fantastic Four,"During a space voyage, four scientists are alt...","[Action, Fantasy, Science Fiction]","[mask, friendship, marriage proposal, fire, si...","[{""cast_id"": 1, ""character"": ""Reed Richards / ...","[{""credit_id"": ""52fe4524c3a36847f80bedd5"", ""de..."
4,166424,Fantastic Four,"During a space voyage, four scientists are alt...","[Action, Fantasy, Science Fiction]","[mask, friendship, marriage proposal, fire, si...","[{""cast_id"": 169, ""character"": ""Reed Richards""...","[{""credit_id"": ""626346b257530e0a7360adfe"", ""de..."
...,...,...,...,...,...,...,...
995,113882,Islander,"After a tragic accident at sea, Eben Cole lose...",[Drama],[],"[{""cast_id"": 4, ""character"": ""Cheryl"", ""name"":...","[{""credit_id"": ""52fe4b40c3a36847f81fa3b5"", ""de..."
996,145549,All Grown Up,Two stories in movie. Story 1 Yukka is in To...,"[Romance, Drama]",[pink film],"[{""cast_id"": 4, ""character"": ""Yuka"", ""name"": ""...","[{""credit_id"": ""584eaf88c3a368322e0109e0"", ""de..."
997,35710,Snakes on a Train,"Under a powerful Mayan curse, snakes are hatch...","[Action, Horror, Thriller]","[drug smuggling, snake, giant snake, hijacking...","[{""cast_id"": 2, ""character"": ""Brujo"", ""name"": ...","[{""credit_id"": ""52fe45b19251416c9103b2bf"", ""de..."
998,14449,The Banquet,Crown Prince Wu Luan is in love with Little Wa...,"[Fantasy, Drama, Action, History]",[],"[{""cast_id"": 1, ""character"": ""Empress Wan"", ""n...","[{""credit_id"": ""52fe45f59251416c75068597"", ""de..."


In [12]:
def convert3(obj):
    L=[]
    k = 0
    for i in ast.literal_eval(obj):
        if(k<3):
            L.append(i['name'])
            k+=1
        else:
            break
    return L

In [13]:
 movies['cast']=movies['cast'].apply(convert3)

In [14]:
def finddir(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if(i['job']=='Director'):
            L.append(i['name'])
            break
    return L

In [15]:
movies['crew']=movies['crew'].apply(finddir)

In [16]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,13499,"Yours, Mine & Ours",Admiral Frank Beardsley returns to New London ...,"[Comedy, Family, Romance]","[big family, family, blended family]","[Dennis Quaid, Rene Russo, Sean Faris]",[Raja Gosnell]
1,411,"The Chronicles of Narnia: The Lion, the Witch ...","Siblings Lucy, Edmund, Susan and Peter step th...","[Adventure, Family, Fantasy]","[witch, epic, sibling relationship, saving the...","[William Moseley, Anna Popplewell, Skandar Key...",[Andrew Adamson]
2,3933,Corpse Bride,"In a 19th-century European village, a young ma...","[Romance, Fantasy, Animation]","[skeleton, love triangle, cheating, wedding vo...","[Johnny Depp, Helena Bonham Carter, Emily Watson]",[Mike Johnson]
3,9738,Fantastic Four,"During a space voyage, four scientists are alt...","[Action, Fantasy, Science Fiction]","[mask, friendship, marriage proposal, fire, si...","[Ioan Gruffudd, Jessica Alba, Chris Evans]",[Tim Story]
4,166424,Fantastic Four,"During a space voyage, four scientists are alt...","[Action, Fantasy, Science Fiction]","[mask, friendship, marriage proposal, fire, si...","[Miles Teller, Michael B. Jordan, Kate Mara]",[Josh Trank]


In [17]:
movies['overview'] = movies['overview'].apply(lambda x:x.split()) #converting string into individual words list

In [18]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [19]:
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [20]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,13499,"Yours, Mine & Ours","[Admiral, Frank, Beardsley, returns, to, New, ...","[Comedy, Family, Romance]","[bigfamily, family, blendedfamily]","[DennisQuaid, ReneRusso, SeanFaris]",[RajaGosnell]
1,411,"The Chronicles of Narnia: The Lion, the Witch ...","[Siblings, Lucy,, Edmund,, Susan, and, Peter, ...","[Adventure, Family, Fantasy]","[witch, epic, siblingrelationship, savingthewo...","[WilliamMoseley, AnnaPopplewell, SkandarKeynes]",[AndrewAdamson]
2,3933,Corpse Bride,"[In, a, 19th-century, European, village,, a, y...","[Romance, Fantasy, Animation]","[skeleton, lovetriangle, cheating, weddingvows...","[JohnnyDepp, HelenaBonhamCarter, EmilyWatson]",[MikeJohnson]
3,9738,Fantastic Four,"[During, a, space, voyage,, four, scientists, ...","[Action, Fantasy, ScienceFiction]","[mask, friendship, marriageproposal, fire, sib...","[IoanGruffudd, JessicaAlba, ChrisEvans]",[TimStory]
4,166424,Fantastic Four,"[During, a, space, voyage,, four, scientists, ...","[Action, Fantasy, ScienceFiction]","[mask, friendship, marriageproposal, fire, sib...","[MilesTeller, MichaelB.Jordan, KateMara]",[JoshTrank]


In [21]:
movies['tags'] = movies['overview'] + movies['keywords'] + movies['cast'] + movies['crew'] + movies['genres']

In [22]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,13499,"Yours, Mine & Ours","[Admiral, Frank, Beardsley, returns, to, New, ...","[Comedy, Family, Romance]","[bigfamily, family, blendedfamily]","[DennisQuaid, ReneRusso, SeanFaris]",[RajaGosnell],"[Admiral, Frank, Beardsley, returns, to, New, ..."
1,411,"The Chronicles of Narnia: The Lion, the Witch ...","[Siblings, Lucy,, Edmund,, Susan, and, Peter, ...","[Adventure, Family, Fantasy]","[witch, epic, siblingrelationship, savingthewo...","[WilliamMoseley, AnnaPopplewell, SkandarKeynes]",[AndrewAdamson],"[Siblings, Lucy,, Edmund,, Susan, and, Peter, ..."
2,3933,Corpse Bride,"[In, a, 19th-century, European, village,, a, y...","[Romance, Fantasy, Animation]","[skeleton, lovetriangle, cheating, weddingvows...","[JohnnyDepp, HelenaBonhamCarter, EmilyWatson]",[MikeJohnson],"[In, a, 19th-century, European, village,, a, y..."
3,9738,Fantastic Four,"[During, a, space, voyage,, four, scientists, ...","[Action, Fantasy, ScienceFiction]","[mask, friendship, marriageproposal, fire, sib...","[IoanGruffudd, JessicaAlba, ChrisEvans]",[TimStory],"[During, a, space, voyage,, four, scientists, ..."
4,166424,Fantastic Four,"[During, a, space, voyage,, four, scientists, ...","[Action, Fantasy, ScienceFiction]","[mask, friendship, marriageproposal, fire, sib...","[MilesTeller, MichaelB.Jordan, KateMara]",[JoshTrank],"[During, a, space, voyage,, four, scientists, ..."


In [23]:
movies = movies[['movie_id','title','tags']]

In [24]:
movies.head()

,movie_id,title,tags
0,13499,"Yours, Mine & Ours","[Admiral, Frank, Beardsley, returns, to, New, ..."
1,411,"The Chronicles of Narnia: The Lion, the Witch ...","[Siblings, Lucy,, Edmund,, Susan, and, Peter, ..."
2,3933,Corpse Bride,"[In, a, 19th-century, European, village,, a, y..."
3,9738,Fantastic Four,"[During, a, space, voyage,, four, scientists, ..."
4,166424,Fantastic Four,"[During, a, space, voyage,, four, scientists, ..."


In [25]:
movies['tags']= movies['tags'].apply(lambda x:" ".join(x))

In [26]:
movies.head()

,movie_id,title,tags
0,13499,"Yours, Mine & Ours",Admiral Frank Beardsley returns to New London ...
1,411,"The Chronicles of Narnia: The Lion, the Witch ...","Siblings Lucy, Edmund, Susan and Peter step th..."
2,3933,Corpse Bride,"In a 19th-century European village, a young ma..."
3,9738,Fantastic Four,"During a space voyage, four scientists are alt..."
4,166424,Fantastic Four,"During a space voyage, four scientists are alt..."


In [27]:
movies['tags']= movies['tags'].apply(lambda x: x.lower())

In [28]:
cv = CountVectorizer(max_features = 20000,stop_words='english')

In [29]:
vector = cv.fit_transform(movies['tags']).toarray()

In [30]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(10384, 20000))

In [31]:
cv.get_feature_names_out()

array(['00', '000', '10', ..., 'ṣọpẹ', '死亡筆記', 'ﬁnd'],
      shape=(20000,), dtype=object)

In [32]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)
    

In [33]:
movies['tags']=movies['tags'].apply(stem)

In [34]:
sim = cosine_similarity(vector)

In [35]:
movies


,movie_id,title,tags
0,13499,"Yours, Mine & Ours",admir frank beardsley return to new london to ...
1,411,"The Chronicles of Narnia: The Lion, the Witch ...","sibl lucy, edmund, susan and peter step throug..."
2,3933,Corpse Bride,"in a 19th-centuri european village, a young ma..."
3,9738,Fantastic Four,"dure a space voyage, four scientist are alter ..."
4,166424,Fantastic Four,"dure a space voyage, four scientist are alter ..."
...,...,...,...
10379,1419229,Silent Zone,"in a zombie-ravag world, a resourc teen and he..."
10380,1010756,The Strangers: Chapter 2,on the final day of their cross countri road t...
10381,1261148,The Deceased,"when their father die on a curs day, terror fo..."
10382,1206083,Conjuring Tapes,while sort through their late friend' belongin...


In [36]:
def recommend(movie):
    index=movies[movies['title']==movie].index[0]
    sim1= sim[index]
    list1 = sorted(list(enumerate(sim1)),reverse=True,key=lambda x:x[1])[1:11]
    for i in list1:
        print(movies.iloc[i[0]].title)

In [ ]:
movies

In [46]:
recommend('Krrish')

Krrish 3
The Thundermans Return
Ra.One
Max Steel
Guardians
LEGO Marvel Avengers: Mission Demolition
The Danger Element
Underdog


In [ ]:
pickle.dump(movies,open('movies.pkl','wb'))

In [ ]:
pickle.dump(sim,open('sim.pkl','wb'))